In [3]:
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from tqdm import tqdm, trange
import pickle
import PIL.Image as Image
import json
import random
import sys
import clip
import PIL
import random


device = torch.device('cuda:0')
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = clip.tokenize
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
_Tokenizer = _Tokenizer()

In [4]:
from torch.utils.data import Dataset, DataLoader
from enum import Enum
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union


        
class MappingType(Enum):
    MLP = 'mlp'
    Transformer = 'transformer'

    
class MLP(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
        

class ClipGpt2Model(nn.Module):

    def __init__(self,prefix_size: int = 512):
        super(ClipGpt2Model, self).__init__()
        with open('./decoder_config.pkl','rb') as f:
            config = pickle.load(f)
        self.decoder = GPT2LMHeadModel(config)
        self.embedding_size = self.decoder.transformer.wte.weight.shape[1]
        self.clip_project = MLP((prefix_size,self.embedding_size))
        
    def forward(self, clip_features,gpt_tokens):
        embedding_text = self.decoder.transformer.wte(gpt_tokens)
        embedding_clip = self.clip_project(clip_features)
        embedding_clip = embedding_clip.reshape(-1,1,self.embedding_size)
        embedding_cat = torch.cat([embedding_clip,embedding_text],dim=1)
        out = self.decoder(inputs_embeds=embedding_cat)
        return out



In [5]:
def Decoding(model,clip_features):
    model.eval()
    embedding_cat = model.clip_project(clip_features).reshape(1,1,-1)
    entry_length = 30
    temperature = 1
    tokens = None
    for i in range(entry_length):
        # print(location_token.shape)
        outputs = model.decoder(inputs_embeds=embedding_cat)

        logits = outputs.logits
        logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
        logits_max = logits.max()
        logits = torch.nn.functional.softmax(logits)
        next_token = torch.argmax(logits, -1).unsqueeze(0)
        next_token_embed = model.decoder.transformer.wte(next_token)

        if tokens is None:
            tokens = next_token

        else:
            tokens = torch.cat((tokens, next_token), dim=1)
        if next_token.item()==49407:
            break
        embedding_cat = torch.cat((embedding_cat, next_token_embed), dim=1)
    try:
        output_list = list(tokens.squeeze().cpu().numpy())
        output = _Tokenizer.decode(output_list)
    except:
        output = 'None'
    return output


In [6]:
model = ClipGpt2Model()
weights_path = './coco_model/coco_prefix-009.pt'
model.load_state_dict(torch.load(weights_path,map_location= torch.device('cpu')))
model = model.to(device)
model = model.eval()

In [8]:
## construct the support memory
with open('./coco_train.json', 'r') as f:
    data = json.load(f)
data = random.sample(data,10000)
text_features = []
captions = []
batch_size = 1000
clip_model.eval()
for i in tqdm(range(0,len(data[:])//batch_size)):
    
    texts = data[i*batch_size:(i+1)*batch_size]
    with torch.no_grad():
        texts_token = tokenizer(texts).to(device)
        text_feature = clip_model.encode_text(texts_token)
        text_features.append(text_feature)
        captions.extend(texts)

text_features = torch.cat(text_features,dim=0)
text_features /= text_features.norm(dim=-1,keepdim=True).float()

100%|██████████| 10/10 [00:04<00:00,  2.27it/s]


In [ ]:
path_pic = '1.png'
image = preprocess(Image.open(path_pic)).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    image_features /= image_features.norm(dim=-1,keepdim=True)
    sim = image_features@text_features.T.float()
    sim = nn.functional.softmax(sim*100)
    prefix_embedding = sim@text_features.float()
    prefix_embedding /= prefix_embedding.norm(dim=-1,keepdim=True)
    generated_text = Decoding(model,prefix_embedding)
    generated_text = generated_text_prefix.replace('<|startoftext|>','').replace('<|endoftext|>','')
    print(generated_text_prefix)